# Entornos

**Notas basadas en:**

* [Advanced R by Hadley Wickham](http://adv-r.had.co.nz/).
* [Learning R - Richard Cotton](http://shop.oreilly.com/product/0636920028352.do).

Todas las variables que creamos se almacenan en un entorno. Los entornos son otro tipo de variable: podemos asignarlos, manipularlos y pasarlos a funciones como argumentos, como se hace  con cualquier otra variable. Los entornos están estrechamente relacionados con las listas que se utilizan para almacenar diferentes tipos de variables. De hecho, la mayoría de la sintaxis de listas también funciona para entornos  y podemos coaccionar una lista para que sea un entorno (y viceversa).

Por lo general, no se necesitará  explícitamente con  los entornos. Por ejemplo, cuando se asigna una variable en la línea de comandos de R, la variable ingresará automáticamente en un entorno denominado `global environment` (también conocido como espacio de trabajo del usuario). Cuando se llama a una función, se crea automáticamente un entorno para almacenar las variables relacionadas con la función. Comprender los conceptos básicos de los entornos puede ser útil para comprender el alcance (*scope*)  de las variables y  examinar la pila de llamadas al depurar el código. El trabajo de un entorno es asociar o unir, un conjunto de nombres a un conjunto de valores. Se podría tener la idea de un entorno como una bolsa de nombres:

![](a1.png)

En R,  los entornos no se crean con la función `environment` (esa función devuelve el entorno que contiene una función en particular). En cambio, la función `new.env` crea un nuevo entorno. Cada nombre apunta a un objeto almacenado en otra parte de la memoria:

In [1]:
e <- new.env()
e$a <- FALSE
e$b <- "a"
e$c <- 2.3
e$d <- 1:3

![](a2.png)

Los objetos no residen  en el entorno, por lo que varios nombres pueden apuntar al mismo objeto:


In [10]:
e$b <- e$d


![](a3.png)

Los nombres también pueden apuntar a objetos diferentes que tienen el mismo valor:

In [12]:
e$b <- 1:3

![](a4.png)

Si un objeto no tiene nombres que apuntan a él, se lo elimina automáticamente con el  recolector de basura.

La asignación de variables a entornos funciona exactamente de la misma manera que con las listas. Se puede utilizar dos corchetes o el operador de signo de dólar. Al igual que con las listas, las variables pueden ser de diferentes tipos y tamaños:

In [32]:
e1 <- new.env()
e1[["r"]] <- c(1, 2, 3, 4, 5)
e1$p <- length(e1[["r"]])

La función `assign` toma como argumento opcional, el entorno que puede ser usado para especificar donde la variable es almacenada.

In [14]:
assign(
    "Jessica",
    20.7,
    e1
)

Recuperar las variables podemos utilizar la sintaxis de ls indexación de listas o utilizar la función `get`, opuesta a `assign`:

In [15]:
e1[["r"]]

[1] 1 2 3 4 5

In [16]:
e1$p

[1] 5

In [17]:
get("Jessica", e1)

[1] 20.7

Las funciones `ls` y `ls.str`  también toman un argumento de entorno, lo que le permite listar su contenido:

In [19]:
ls(env = e)

[1] "a" "b" "c" "d"

In [20]:
ls (env =e1)

[1] "Jessica" "p"       "r"

In [21]:
ls.str(envir = e)

a :  logi FALSE
b :  int [1:3] 1 2 3
c :  num 2.3
d :  int [1:3] 1 2 3

In [22]:
ls.str(envir = e1)

Jessica :  num 20.7
p :  int 5
r :  num [1:5] 1 2 3 4 5

Cada entorno  tiene un padre, otro entorno (Hadley  Wickham, usa la metáfora de una familia para referirse a entornos) . En el gráfico, se representa un  puntero al entorno padre con un pequeño círculo negro. El entorno padre se utiliza entre otras cosas  para implementar el **alcance léxico**: si no se encuentra un nombre en un entorno, entonces R buscará en su entorno padre y así sucesivamente . Sólo un entorno no tiene un padre: el entorno vacío.

![](a5.png)

Generalmente, un entorno es similar a una lista, con cuatro excepciones importantes:

* Cada nombre en un entorno es único.
* Los nombres en un entorno no está ordenados (no tiene sentido preguntar, cual es el primer elemento de un entorno).
* Un entorno tiene un padre: 
* Los entornos tienen referencias semánticas.

Técnicamente, un entorno se compone de dos componentes, el `frame`,  que contiene los enlaces `nombre-objeto` y se comporta como una lista con nombres  y un entorno padre.

Hay cuatro entornos especiales:

* `globalenv()` o entorno global,  es el espacio de trabajo interactivo. El padre del entorno global es el último paquete que se ajuntó  con `library ()` o `require()`. Este es el entorno en el que normalmente  se trabaja.
* `baseenv()` , o entorno base  es el entorno del paquete `base`. Su padre es el entorno vacío
* `emptyenv()`, es el predecesor  de todos los entornos  y el único entorno sin un padre.
* `environment` () es el entorno actual.



`search()` muestra todos los padres del entorno global. Se le llama   `ruta de búsqueda` porque los objetos en estos entornos se pueden encontrar desde el espacio de trabajo interactivo de nivel superior. Contiene un entorno para cada paquete adjunto y cualquier otro objeto que se haya  adjuntado. También contiene un entorno especial llamado `Autoloads` que se utiliza para guardar memoria al cargar objetos de un paquete (como grandes conjuntos de datos) cuando sea necesario.

In [29]:
search()

[1] ".GlobalEnv"        "jupyter:irkernel"  "jupyter:irkernel" 
 [4] "package:stats"     "package:graphics"  "package:grDevices"
 [7] "package:utils"     "package:datasets"  "package:methods"  
[10] "Autoloads"         "package:base"

Puede acceder a cualquier entorno de la lista de búsqueda utilizando `as.environment()`:

In [27]:
as.environment("jupyter:irkernel")

<environment: 0x2ab6bc8>
attr(,"name")
[1] "jupyter:irkernel"

In [30]:
as.environment("package:stats")

<environment: package:stats>
attr(,"name")
[1] "package:stats"
attr(,"path")
[1] "/home/c-lara/anaconda3/lib/R/library/stats"

`globalenv ()`, `baseenv ()`, los entornos entre   la ruta de búsqueda y `emptyenv()` están conectados como se muestra a continuación. Cada vez que se carga un nuevo paquete con `library()` se inserta entre el entorno global y el paquete que estaba previamente en la parte superior de la ruta de búsqueda.

![](a6.png)

In [39]:
# Ejemplo de resumen

e2 <- new.env()
parent.env(e2)
ls(e2)

# Modifiquemos las uniones en un entornp

e2$a <- "R"
e2$b <- 2
ls(e2)

e2$a
e2$b

ls(e2, all.names = TRUE)

# Vemos el contenido de un archivo

str(e2)
ls.str(e2)

<environment: R_GlobalEnv>

character(0)

[1] "a" "b"

[1] "R"

[1] 2

[1] "a" "b"

<environment: 0x1b918d0> 


a :  chr "R"
b :  num 2

In [45]:
# Uso de [[ $ o get()

e2$c <- 3
e2$c
e2$d <- "R good!"
e2[["d"]]
e2$g <- 4.5

get("g", envir = e2)


[1] 3

[1] "R good!"

[1] 4.5

Eliminar algún  objeto  en un  entornos funciona un poco diferente de lo que sucede con las listas. En una lista puedes remover una entrada , con   `NULL`. En en el caso de un entorno, se creará un nuevo enlace a `NULL`. En su lugar, utilizamos `rm ()`  para quitar el enlace.

In [46]:
e3 <- new.env()

e3$a <- 'Jessica'
e3$a <- NULL
ls(e3)

[1] "a"

In [48]:
# Eliminando el entorno

rm("a", envir = e3)
ls(e3)

Warning message in rm("a", envir = e3):
“objeto 'a' no encontrado”

character(0)

Todos los entornos están anidados, lo que significa que deben tener un entorno padre (como se dijo la  excepción aquí es un entorno especial llamado entorno vacío que se encuentra en la parte superior de la cadena). De forma predeterminada, las funciones `exists` y `get` también buscarán variables en los entornos padres. Pasamos `inherits = FALSE`  para cambiar este comportamiento de modo que sólo se buscará en el entorno que se ha especificado.

In [49]:
x <- 10
exists("x", envir = e3)

[1] TRUE

In [50]:
exists("x", envir = e3, inherits = FALSE)

[1] FALSE

In [53]:
e1_anidado <- new.env(parent = e1)
exists('r', e1_anidado) 

[1] TRUE

In [54]:
exists("r", e1_anidado, inherits = FALSE)

[1] FALSE

Para comparar entornos se usa `identical()` en lugar de `==`:

In [51]:
globalenv() == environment()

ERROR: Error in globalenv() == environment(): comparación  (1) es posible solo para tipos lista y atómico


In [52]:
identical(globalenv(), environment())

[1] TRUE

Las funciones de acceso directo están disponibles para acceder al entorno global (donde se almacenan las variables que se asignan desde el símbolo del sistema) y al entorno base (contiene funciones y otras variables del paquete base de R, que proporciona funcionalidad básica):

In [65]:
# Ejemplo de una asignacion global

n1 <<- c(3, 7, 8, 13, 17, 18, 21) 
get("n1", envir = globalenv())

[1]  3  7  8 13 17 18 21

In [63]:
head(ls(envir = baseenv()), 25)

[1] "-"                 "-.Date"            "-.POSIXt"         
 [4] ":"                 "::"                ":::"              
 [7] "!"                 "!.hexmode"         "!.octmode"        
[10] "!="                "("                 "["                
[13] "[.AsIs"            "[.data.frame"      "[.Date"           
[16] "[.difftime"        "[.Dlist"           "[.factor"         
[19] "[.hexmode"         "[.listof"          "[.noquote"        
[22] "[.numeric_version" "[.octmode"         "[.POSIXct"        
[25] "[.POSIXlt"

Hay otras dos situaciones en las que podemos encontrar entornos. En primer lugar, cuando se llama a una función, todas las variables definidas por la función se almacenan en un entorno que pertenece a esa función (una función más su entorno se denomina a veces `closure`). En segundo lugar, cuando cargamos un paquete, las funciones de ese paquete se almacenan en un entorno en la ruta de búsqueda.